## Modeling and Simulation in Python

Project 3

Jinfay Yuan and Manu Patil

In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *
import math
import matplotlib.pyplot as plt

from IPython.display import HTML

import array as arr
import matplotlib.pyplot as plt
import matplotlib.animation
np.random.seed(7)

In [ ]:
N = UNITS.newton
kg = UNITS.kilogram
AU = UNITS.astronomical_unit
day = UNITS.day
m = UNITS.meter
s = UNITS.second
y = UNITS.year
degree = UNITS.degree

In [ ]:
params = Params(Mass_Earth = 5.972e24 *kg,
                Mass_Moon = 7.3476730e22 *kg, 
                Mass_Satellite = 100 *kg,
                G = 6.674e-11* (N*m**2)/(kg**2),
                r_0 = 3449709330,#distance from center of Earth to Lagrange point
                t_0 = 0*s,
                t_end =1000*s,
                v=10000*m/s,
                theta=30*degree)

In [ ]:
def make_system(params):
    unpack(params)


    init = State(x = 344970933 *m, 
                 y = 0*m ,
                 vx = v* math.cos(theta) *m/s ,
                 vy = v* math.sin(theta) *m/s ,
                 mx = 384472281 *m, #distance from Earth to Moon
                 my = 0*m,
                 mvx = 0 *m/s,
                 mvy = 1000 * m/s);

    return System(params,init=init)
system = make_system(params)

In [ ]:
def slope_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of y and v
    """
    x, y, vx, vy, mx, my, mvx, mvy = state
    unpack(system) 
    
    forceSatellite = Earth_fgrav(state,system) + Moon_fgrav(state,system)
    forceEarthMoon = Earth_fgravMoon(state,system)
    
    dxdt = vx 
    dydt = vy
    
    dvxdt = (forceSatellite.x) / Mass_Satellite
    dvydt = (forceSatellite.y) / Mass_Satellite
    
    dmxdt = mvx
    dmydt = mvy
    
    dmvxdt = (forceEarthMoon.x) / Mass_Moon
    dmvydt = (forceEarthMoon.y) / Mass_Moon
    

    
    return dxdt, dydt, dvxdt, dvydt,dmxdt, dmydt, dmvxdt, dmvydt

In [ ]:
def fgrav(obj1_x, obj1_y, obj2_x, obj2_y, Mass_1, Mass_2):
    position = Vector(obj2_x-obj1_x, obj2_y-obj1_y) *m
    force = G * Mass_2 * Mass_1 / (position.mag)**2
    return  position.hat() * force

In [ ]:
def Earth_fgravMoon(state,system):
    unpack (system)
    x,y,vx,vy,mx,my,mvx,mvy = state
    return fgrav(mx,my, 0, 0, Mass_Moon, Mass_Earth)
Earth_fgravMoon(system.init, system)

In [ ]:
def Earth_fgrav(state,system):
    unpack (system)
    x,y,vx,vy,mx,my,mvx,mvy = state
    return fgrav(x,y, 0, 0, Mass_Satellite, Mass_Earth)
Earth_fgrav(system.init, system)

In [ ]:
def Moon_fgrav(state,system):
    unpack (system)
    x,y,vx,vy,mx,my,mvx,mvy = state
    return fgrav(x,y, mx, my, Mass_Satellite, Mass_Moon)
Moon_fgrav(system.init, system)

In [ ]:
def event_func(state, t, system): # Checks crash into Moon
    unpack (system)
    x,y,vx,vy,mx,my,mvx,mvy = state
    position = Vector(mx-x, my-y) *m
    return position.mag /m - 1737000*m
event_func(system.init, 0, system)

In [ ]:
def event_func1(state, t, system): # Checks crash into Earth
    unpack (system)
    x,y,vx,vy,mx,my,mvx,mvy = state
    position = Vector(0-x, 0-y) *m
    return position.mag /m - 6371000*m
event_func1(system.init, 0, system)

In [ ]:
params = Params(params, v = 1000*m, theta = 45 * degree, t_end = 1e5*s)
system = make_system(params)
slope_func(system.init, 0, system)

In [ ]:
params = Params(params, v = 1000*m/s, theta = 180 * degree, t_end = 1e6*s)
system = make_system(params)
results, details = run_ode_solver(system, slope_func,events = [event_func,event_func1], method = "LSODA")
details

In [ ]:
def plot_position(results):
    unpack(system)
    count = 1
    
 
    subplot(2,1,1) # Plots x and y over time. Not terribly useful
    for result in results:
        result.index /= 60*60*24 #convert seconds to days

        plot(result.x, label='x'+str(count))
        plot(result.y, label = 'y'+ str(count))
        count +=1
    decorate(xlabel='Time (second)',
             ylabel='Distance from earth (million m)')
    
    subplot(2,1,2) #Plots x against y creating 
    count = 1
    fig = plt.gcf()
    ax = fig.gca()
    circle1 = plt.Circle((384472281, 0), 1736482, color='g',label = "Moon Start")
    circle2 = plt.Circle((0, 0), 6371000, color='b', label = "Earth")
    ax.add_artist(circle1)
    ax.add_artist(circle2)
    for result in results:
        plot(result.x,result.y, label = 'Position'+str(count))
        plot(result.mx,result.my, label = 'Moon Path' + str(count))
        circle = plt.Circle((get_last_value(result.mx), get_last_value(result.my)), 1736482, color='r',label = "Moon End"+str(count))
        ax.add_artist(circle)
        count +=1

    decorate(xlabel='x Distance from earth (m)',
             ylabel='y Distance from earth (m)')
plot_position([results])

In [ ]:
params = Params(params, v = 500*m/s, theta = -30 * degree, t_end = 2100000*s)
system = make_system(params)
results1, details = run_ode_solver(system, slope_func, events = [event_func,event_func1], method = "LSODA")


params = Params(params, v = 500*m/s, theta = 30 * degree, t_end = 2100000*s)
system = make_system(params)
ts = linspace(0,system.t_end, 500)
results2, details = run_ode_solver(system, slope_func, events = [event_func,event_func1], method = "LSODA", t_eval = ts)
plot_position([results1,results2])

In [ ]:
def Distance_func(angle, params):  
    """Computes range for a given launch angle.
    
    angle: launch angle in degrees
    params: Params object
    
    returns: distance in meters
    """
    
    print("here")
    params = Params(params, theta = params.theta + angle*degree, t_end = 1e6*s, v = params.v + 100*m/s)
    print(params)
    system = make_system(params)
    print(system)
    
    results, details = run_ode_solver(system, slope_func, events = [event_func,event_func1], method = "LSODA")
    Earth_dist = Vector(get_last_value(results.x), get_last_value(results.y))*m
    print(system.theta, Earth_dist.mag)
    return Earth_dist.mag

In [ ]:
Distance_func(45,params)

In [ ]:
res = min_bounded(Distance_func, [-80, 90], params)

In [ ]:
res.x

In [ ]:
params = Params(params, theta=res.x*degree)
system = make_system(params)
print(system.theta)
results7, details = run_ode_solver(system, slope_func, events=[event_func,event_func1], method = "LSODA")
print(details)
plot_position([results7])

Course Corrections

In [ ]:
# Add Vector in direction in random direction in a range of velocties.

In [ ]:
def make_system2(mEarth, mMoon, mSatellite, G, r_0, v,theta, t_end):

    rad = (theta/180) * pi
    
    init = State(x = 344970933 *m, 
                 y = 0*m ,
                 vx = v* math.cos(rad) *m/s ,
                 vy = v* math.sin(rad) *m/s ,
                 xi = 344970933 *m, 
                 yi = 0*m ,
                 vxi = v* math.cos(rad) *m/s ,
                 vyi = v* math.sin(rad) *m/s ,
                 mx = 384472281 *m, #distance from Earth to Moon
                 my = 0*m,
                 mvx = 0 *m/s,
                 mvy = 1000 * m/s);

    
    t_0 = 0 *s
    t_end = t_end *s
    
    
    return System(init=init,
                mEarth = mEarth,
                mMoon = mMoon,
                mSatellite = mSatellite,
                G = G,
                t_0 = t_0,
                t_end=t_end)
system = make_system2(Mass_Earth, Mass_Moon, Mass_Satellite, G, r_0, 100, 100, 1000)

In [ ]:
def slope_func2(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of y and v
    """
    x, y, vx, vy, xi, yi, vxi, vyi, mx, my, mvx, mvy = state
    unpack(system) 
    
    forceSatellite = Earth_fgrav(state,system) + Moon_fgrav(state,system)
    forceEarthMoon = Earth_fgravMoon(state,system)
    forceSatellitei = Earth_fgrav_ideal(state,system) + Moon_fgrav_ideal(state,system)

    if t == 0:
        explosion =  forceOxygen_Explosion()
        print(explosion)
    else:
        explosion = Vector(0,0) *m;
    
    dxdt = vx 
    dydt = vy
    
    dvxdt = (forceSatellite.x + explosion.x) / Mass_Satellite
    dvydt = (forceSatellite.y + explosion.y) / Mass_Satellite
    
    dxidt = vxi 
    dyidt = vyi
    
    dvxidt = (forceSatellitei.x) / Mass_Satellite
    dvyidt = (forceSatellitei.y) / Mass_Satellite
    
    dmxdt = mvx
    dmydt = mvy
    
    dmvxdt = (forceEarthMoon.x) / Mass_Moon
    dmvydt = (forceEarthMoon.y) / Mass_Moon
    

    print(dvxdt == dvxidt)
    return dxdt, dydt, dvxdt, dvydt, dxidt, dyidt, dvxidt, dvyidt, dmxdt, dmydt, dmvxdt, dmvydt

In [ ]:
def forceOxygen_Explosion():
    direction = np.random.random()*360 * degree
    acc = (np.random.random()*10000 + 10000) * m/s**2
    force = Vector(acc*math.cos(direction),acc*math.sin(direction))*m
    return force
    
print(forceOxygen_Explosion())

In [ ]:
def event_func3(state, t, system): # Checks crash into Earth
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi,mx,my,mvx,mvy = state
    position = Vector(0-x, 0-y) *m
    return position.mag /m - 6371000*m
event_func1(system.init, 0, system)

In [ ]:
def event_func4(state, t, system): # Checks crash into Moon
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi,mx,my,mvx,mvy = state
    position = Vector(mx-x, my-y) *m
    return position.mag /m - 1737000*m
event_func(system.init, 0, system)

In [ ]:
def Earth_fgravMoon(state,system):
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi,mx,my,mvx,mvy = state
    return fgrav(mx,my, 0, 0, Mass_Moon, Mass_Earth)
Earth_fgravMoon(system.init, system)

def Earth_fgrav(state,system):
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi,mx,my,mvx,mvy = state
    return fgrav(x,y, 0, 0, Mass_Satellite, Mass_Earth)
Earth_fgrav(system.init, system)

def Moon_fgrav(state,system):
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi, mx,my,mvx,mvy = state
    return fgrav(x,y, mx, my, Mass_Satellite, Mass_Moon)
Moon_fgrav(system.init, system)

In [ ]:
def Earth_fgrav_ideal(state,system):
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi,mx,my,mvx,mvy = state
    return fgrav(xi,yi, 0, 0, Mass_Satellite, Mass_Earth)
Earth_fgrav(system.init, system)

def Moon_fgrav_ideal(state,system):
    unpack (system)
    x,y,vx,vy, xi, yi, vxi, vyi, mx,my,mvx,mvy = state
    return fgrav(xi,yi, mx, my, Mass_Satellite, Mass_Moon)
Moon_fgrav(system.init, system)

In [ ]:
system = make_system2(Mass_Earth, Mass_Moon, Mass_Satellite, G, r_0, 500,30,2100000)
results3, details = run_ode_solver(system, slope_func2, events = [event_func,event_func1], method = "LSODA")

In [ ]:
def plot_position2(results):
    unpack(system)
    count = 1
    
 
    subplot(2,1,1) # Plots x and y over time. Not terribly useful
    for result in results:
        result.index /= 60*60*24 #convert seconds to days
#         result.x /= 1e9
#         result.y /= 1e9 #converts to millions of km
#         result.mx /= 1e9
#         result.my /= 1e9
        plot(result.x, label='x'+str(count))
        plot(result.y, label = 'y'+ str(count))
        plot(result.xi, label='Ideal x'+str(count))
        plot(result.yi, label = 'Ideal y'+ str(count))
        count +=1
    decorate(xlabel='Time (second)',
             ylabel='Distance from earth (million m)')
    
    subplot(2,1,2) #Plots x against y creating 
    count = 1
    fig = plt.gcf()
    ax = fig.gca()
    circle1 = plt.Circle((384472281, 0), 1736482, color='g',label = "Moon Start")
    circle2 = plt.Circle((0, 0), 6371000, color='b', label = "Earth")
    ax.add_artist(circle1)
    ax.add_artist(circle2)
    for result in results:
        plot(result.x,result.y,"r", label = 'Position'+str(count),)
        plot(result.xi,result.yi,":r", label = 'Ideal Position'+str(count))
        #plot(result.mx,result.my, label = 'Moon Path' + str(count))
        circle = plt.Circle((get_last_value(result.mx), get_last_value(result.my)), 1736482, color='r',label = "Moon End"+str(count))
        ax.add_artist(circle)
        count +=1

    decorate(xlabel='x Distance from earth (m)',
             ylabel='y Distance from earth (m)')
plot_position2([results3])

### Animation Stuffs

In [ ]:
print(results2.mx[results2.index[10]])


In [ ]:
def anim(results):

    plt.rcParams["animation.html"] = "jshtml"
    t = results

    fig, ax = plt.subplots();
    ax.axis([-4e8, 4e8, -4e8, 4e8]);
    circle1 = plt.Circle((384472281, 0), 1736482, color='g',label = "Moon Start")
    circle2 = plt.Circle((0, 0), 6371000, color='b', label = "Earth")
    ax.add_artist(circle1)
    ax.add_artist(circle2)
    l, = ax.plot([],[]);
    l1, = ax.plot([],[]);
    #l2, = ax.plot([],[]);

    def animate(i):
        l.set_data(results.mx[results.index[:i]], results.my[results.index[:i]]);
        l1.set_data(results.x[results.index[:i]], results.y[results.index[:i]]);
        #l2.set_data(results.xi[results.index[:i]], results.yi[results.index[:i]]);

    ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t.index));
    print(len(t.index))
    HTML(ani.to_jshtml())
    return ani

ani = anim(results2)

In [ ]:
HTML(ani.to_jshtml())